# Model Based Grading

## Key Concepts
- Grading provides objective quality signals (typically 1-10 scale, 10 = highest quality)
- Three grader types: code graders (programmatic checks), model graders (AI evaluation), human graders (manual review)
- Model graders use additional API call to evaluate original output
- Define evaluation criteria before implementing graders (format, syntax, task following)
- Request strengths/weaknesses/reasoning alongside score to avoid middling defaults
- Calculate average score across all test cases for objective metric
- Model graders provide flexible assessment but can be inconsistent

## Important Code Patterns
- `grade_by_model(test_case, output)` - sends evaluation prompt to Claude
- Use structured JSON output with pre-fill: `add_assistant_message(messages, "```json")`
- Include evaluation criteria in prompt: strengths, weaknesses, reasoning, score
- `json.loads(eval_text)` - parse grader response
- `mean([result["score"] for result in results])` - calculate average score
- Update `run_test_case()` to call grad

In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [4]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [5]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [6]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""

    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [7]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [12]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [13]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 5.333333333333333


In [10]:
print(json.dumps(results, indent=2))

[
  {
    "output": "# Parse AWS S3 Bucket Name from URI\n\nHere's a solution with multiple approaches:\n\n## 1. Simple String Parsing (Python)\n\n```python\ndef parse_s3_bucket_name(s3_uri):\n    \"\"\"Extract bucket name from S3 URI\"\"\"\n    # Remove s3:// prefix and get the first part before the first /\n    bucket_name = s3_uri.replace(\"s3://\", \"\").split(\"/\")[0]\n    return bucket_name\n\n# Test\nuri = \"s3://my-bucket-name/key/path/file.txt\"\nprint(parse_s3_bucket_name(uri))  # Output: my-bucket-name\n```\n\n## 2. Using Regex (Python)\n\n```python\nimport re\n\ndef parse_s3_bucket_name(s3_uri):\n    \"\"\"Extract bucket name using regex\"\"\"\n    match = re.match(r\"s3://([^/]+)\", s3_uri)\n    return match.group(1) if match else None\n\n# Test\nuri = \"s3://my-bucket-name/key/path\"\nprint(parse_s3_bucket_name(uri))  # Output: my-bucket-name\n```\n\n## 3. Using urllib (Python)\n\n```python\nfrom urllib.parse import urlparse\n\ndef parse_s3_bucket_name(s3_uri):\n    \"\"